<a href="https://colab.research.google.com/github/ogigo/Text_Summaeization/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
api_token = {"username":"shahedshoyab","key":"1aa2d9748604f09744dac84eb27b107f"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:json.dump(api_token, file)

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download gowrishankarp/newspaper-text-summarization-cnn-dailymail

 99% 500M/503M [00:24<00:00, 24.2MB/s]
100% 503M/503M [00:25<00:00, 21.1MB/s]


In [ ]:
!unzip newspaper-text-summarization-cnn-dailymail.zip

Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.0 MB/s 
     |████████████████████████████████| 7.6 MB 68.6 MB/s 
     |████████████████████████████████| 182 kB 64.7 MB/s 


In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 24.0 MB/s 
     |████████████████████████████████| 512 kB 65.2 MB/s 
     |████████████████████████████████| 125 kB 58.2 MB/s 


In [ ]:
import seaborn as sns
import pandas as pd 
from transformers import AdamW, T5ForConditionalGeneration, T5TokenizerFast as T5Tokenizer
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import textwrap
from pathlib import Path

import seaborn as sns
from tqdm.auto import tqdm
from pylab import rcParams
import matplotlib.pyplot as plt 
from matplotlib import rc 
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
pl.seed_everything(42)

INFO:lightning_lite.utilities.seed:Global seed set to 42


42

In [ ]:
train_df=pd.read_csv("/content/cnn_dailymail/train.csv")

In [ ]:
train_df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
train_df=train_df[1:10000]

In [ ]:
train,test=train_test_split(train_df,test_size=0.2)

In [ ]:
train.reset_index(drop=True)

,id,article,highlights
0,1a3aa5e5f40f0d8dab27b739a5ba6bae7ec32cef,Floyd Mayweather Jnr has named May 2 as the da...,Floyd Mayweather looks set to finally fight Ma...
1,047c35f11915c6fd24706d9fb12fddfbc4b1e25e,Stilyan Petrov is back with a ball at his feet...,Stiliyan Petrov was diagnosed with leukemia in...
2,04c6d9e9cbd82a735831aa10aabf9060b62993d7,While Rory McIlroy was forced to put his champ...,Australian Wade Ormsby leads Turkish Airlines ...
3,1140f84385d482dd3987635068ef4d0a815a0dd4,Entrepreneurial Maccas fans have jumped on the...,A 25ml sized tub of Big Mac sauce is now selli...
4,14c38ce9078967567b2101333c613a540ee351b5,"By . Sean Poulter . PUBLISHED: . 09:35 EST, 16...",French fashion house repositioning themselves ...
...,...,...,...
7994,104a2d5c4fe33c8c98804f6440e4286f167aa838,By . Luke Augustus . Follow @@Luke_Augustus29 ...,Antonio Conte has left Juventus by mutual cons...
7995,0eb8e5740eecc2098cd862cb5d1ff41f9aa97eb5,A starving four-month-old foal was left to die...,Gizmo is fighting for his life after he was re...
7996,0f4b521ea8984f50b3fbdd306f7dddac332a7620,"Out of England's World Cup squad, out of contr...","Ashley Cole's contract is up at Chelsea, but h..."
7997,0276ba238b029b225b43e657dc69b2847d1479a2,By . Daily Mail Reporter . PUBLISHED: . 05:59 ...,CCTV footage shows soldiers filling shopping b...


In [ ]:
test.reset_index(drop=True)

,id,article,highlights
0,0f2025d9759060f141cb1a36aa7d54bdcc4c0139,By . Daily Mail Reporter and Sam Shead . PUBLI...,Long Endurance Multi-Intelligence Vehicle (LEM...
1,1518803f30c09bd21e45ec1a6c67296b17b49bf5,"By . Anna Edwards . PUBLISHED: . 05:57 EST, 29...","Father-of-two escaped with only whiplash, shoc..."
2,04fb72dc5ae9ac6dc9c85dc55211fdea67896e2c,By . Associated Press . PUBLISHED: . 00:11 EST...,"After 13 years on the run, man was caught urin..."
3,1889e0d55f1e2c634b3206fccf6eb751c570eeb6,(CNN) -- Holders Brazil were knocked out of th...,Star-studded Brazil missed all four of their s...
4,0d03c47d1c61ee585da8cbdf7d7e6f384007d530,"(CNN) -- Shirley Chisholm, elected in 1968 as...","Women's progress in this election ""creeping an..."
...,...,...,...
1995,1ac8d61acdbe81547a181156d9161f2e94e48c4d,Thomas Eric Duncan's temperature spiked to 103...,Thomas Duncan's family provided his medical re...
1996,17701b9f238db96a3488d6d71a8a92cac4649d5e,The European High Court ruled that a person ha...,"European court ruled that you have a ""right to..."
1997,163b7d4ec24551c270f4bff7b4055c04f590be83,"ATLANTA, Georgia (CNN) -- Three men who kidnap...",3 men linked to cartels kidnapped and tortured...
1998,031bc34be6b4ca7139836a16b1d9d1b616c72a65,(CNN) -- A Qantas passenger jet landed safely ...,NEW: Passenger says he saw flames outside the ...


In [ ]:
class NewsSummaryDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        text_max_token_len: int = 512,
        summary_max_token_len: int = 128):
        
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index:int):
        data_row = self.data.iloc[index]
        text = data_row["article"]
        
        text_encoding = tokenizer(text,max_length=self.text_max_token_len,
                                 padding="max_length", 
                                 truncation=True,
                                 return_attention_mask=True,
                                 add_special_tokens=True,
                                 return_tensors="pt")
        
        summary = data_row["highlights"]
        summary_encoding = tokenizer(summary,max_length=self.summary_max_token_len,
                                 padding="max_length", 
                                 truncation=True,
                                 return_attention_mask=True,
                                 add_special_tokens=True,
                                 return_tensors="pt")
        
        labels= summary_encoding["input_ids"] 
        labels[labels == 0] = -100
        
        return dict(
            text=text, 
            summary=summary,
            text_input_ids=text_encoding["input_ids"].flatten(),
            text_attention_mask=text_encoding["attention_mask"].flatten(),
            labels=labels.flatten(),
            labels_attention_mask=summary_encoding["attention_mask"].flatten())

In [ ]:
class NewsSummaryDataModule(pl.LightningDataModule):
    def __init__(self, 
                train_df:pd.DataFrame,
                test_df:pd.DataFrame,
                tokenizer:T5Tokenizer,
                batch_size: int = 8,
                text_max_token_len: int = 512,
                summary_max_token_len: int = 128):
        super().__init__()
        self.train_df=train_df
        self.test_df=test_df
        
        self.batch_size=batch_size
        self.tokenizer=tokenizer
        self.text_max_token_len=text_max_token_len
        self.summary_max_token_len= summary_max_token_len
    
    def setup(self, stage=None):
        self.train_dataset =  NewsSummaryDataset(
            self.train_df,
            self.tokenizer,
            self.text_max_token_len,
            self.summary_max_token_len
        )
        self.test_dataset =  NewsSummaryDataset(
            self.test_df,
            self.tokenizer,
            self.text_max_token_len,
            self.summary_max_token_len
        )
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=2)
    
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2)
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2)

In [ ]:
MODEL_NAME = "t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
class NewsSummaryModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        
    def forward(self, inputs_ids, attention_mask, decoder_attention_mask, labels=None):
        output = self.model(inputs_ids,
                            attention_mask=attention_mask,
                            labels=labels,
                            decoder_attention_mask=decoder_attention_mask)
        return output.loss, output.logits
    
    def step(self, batch, batch_idx):
        input_ids=batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels=batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]
        
        loss, outputs = self.forward(inputs_ids=input_ids,
                             attention_mask=attention_mask,
                             decoder_attention_mask=labels_attention_mask,
                             labels=labels)
        return loss, outputs
    
    def training_step(self, batch, batch_idx):
        loss, outputs = self.step(batch, batch_idx)
        
        self.log("train_loss",loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss, outputs = self.step(batch, batch_idx)
        self.log("val_loss",loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        loss, outputs = self.step(batch, batch_idx)
        self.log("test_loss",loss, prog_bar=True, logger=True)
        return loss
    
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)

In [ ]:
model = NewsSummaryModel()

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
checkpoint_callback = ModelCheckpoint(
                        dirpath="checkpoints",
                        filename="best-checkpoint",
                        save_top_k=1,
                        verbose=True, 
                        monitor="val_loss",
                        mode="min")
logger = TensorBoardLogger("lightning_logs", name="news-summary")


trainer = pl.Trainer(logger=logger,
                    enable_checkpointing=checkpoint_callback,
                    max_epochs=5,
                    gpus=1)


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
N_EPOCHS = 3
BATCH_SIZE = 8

data_module = NewsSummaryDataModule(train,test,tokenizer,batch_size=BATCH_SIZE)

In [ ]:
trainer.fit(model,data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]